In [16]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle (1).json


In [17]:
!kaggle datasets download -d kashnitsky/hierarchical-text-classification

hierarchical-text-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [18]:
!unzip *.zip

Archive:  hierarchical-text-classification.zip
replace train_40k.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_40k.csv           
replace unlabeled_150k.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace val_10k.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [66]:
import pandas as pd
import numpy as np

import string
import re

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
data = pd.read_csv('train_40k.csv')

In [21]:
data.head()

,productId,Title,userId,Helpfulness,Score,Time,Text,Cat1,Cat2,Cat3
0,B000E46LYG,Golden Valley Natural Buffalo Jerky,A3MQDNGHDJU4MK,0/0,3.0,-1,The description and photo on this product need...,grocery gourmet food,meat poultry,jerky
1,B000GRA6N8,Westing Game,unknown,0/0,5.0,860630400,This was a great book!!!! It is well thought t...,toys games,games,unknown
2,B000GRA6N8,Westing Game,unknown,0/0,5.0,883008000,"I am a first year teacher, teaching 5th grade....",toys games,games,unknown
3,B000GRA6N8,Westing Game,unknown,0/0,5.0,897696000,I got the book at my bookfair at school lookin...,toys games,games,unknown
4,B00000DMDQ,I SPY A is For Jigsaw Puzzle 63pc,unknown,2/4,5.0,911865600,Hi! I'm Martine Redman and I created this puzz...,toys games,puzzles,jigsaw puzzles


In [0]:
data.drop(data.columns[:6] , inplace = True , axis = 1)

In [23]:
data.head()

,Text,Cat1,Cat2,Cat3
0,The description and photo on this product need...,grocery gourmet food,meat poultry,jerky
1,This was a great book!!!! It is well thought t...,toys games,games,unknown
2,"I am a first year teacher, teaching 5th grade....",toys games,games,unknown
3,I got the book at my bookfair at school lookin...,toys games,games,unknown
4,Hi! I'm Martine Redman and I created this puzz...,toys games,puzzles,jigsaw puzzles


In [24]:
data.Cat1.value_counts()

toys games              10266
health personal care     9772
beauty                   5846
baby products            5637
pet supplies             4862
grocery gourmet food     3617
Name: Cat1, dtype: int64

In [25]:
data.Cat2.value_counts()

personal care         2852
dogs                  2610
nutrition wellness    2203
health care           2002
cats                  1779
                      ... 
sauces dips             40
baby food               40
produce                 38
meat seafood            37
small animals           29
Name: Cat2, Length: 64, dtype: int64

In [26]:
data.Cat2.value_counts().shape

(64,)

In [27]:
data.shape

(40000, 4)

In [28]:
data.Cat3.value_counts()

unknown                 2262
shaving hair removal    1565
vitamins supplements    1315
board games              924
styling tools            850
                        ... 
children s                 1
bubble bath                1
butter                     1
odor stain removers        1
pork                       1
Name: Cat3, Length: 464, dtype: int64

In [29]:
table = str.maketrans('' , '' , string.punctuation)
def clean(txt):
    stripped = [w.translate(table) for w in txt.split()]
    txt = ' '.join([words for words in stripped])
    result = ''.join([i for i in txt if not i.isdigit()])
    words = [word.lower() for word in result.split()]
    stop = set(stopwords.words('english'))
    wo = [word for word in words if not word in stop]
    txt = ' '.join(wo)
    return txt
print(type(clean(data.Text[1])))

<class 'str'>


In [30]:
data.Text[1]

'This was a great book!!!! It is well thought through, and you can easily imagine the events happening. The Westing Game itself is a great way to tie two things together. The events are well sequenced and exciting. Ellen Raskin wrote a wonderful book'

In [0]:
data["new_text"] = data["Text"].apply(lambda x : clean(x))

In [32]:
data.head()

,Text,Cat1,Cat2,Cat3,new_text
0,The description and photo on this product need...,grocery gourmet food,meat poultry,jerky,description photo product needs changed indica...
1,This was a great book!!!! It is well thought t...,toys games,games,unknown,great book well thought easily imagine events ...
2,"I am a first year teacher, teaching 5th grade....",toys games,games,unknown,first year teacher teaching th grade special r...
3,I got the book at my bookfair at school lookin...,toys games,games,unknown,got book bookfair school looking something sum...
4,Hi! I'm Martine Redman and I created this puzz...,toys games,puzzles,jigsaw puzzles,hi im martine redman created puzzle briarpatch...


In [0]:
data.drop('Text' , axis = 1 , inplace = True)

In [34]:
data.head()

,Cat1,Cat2,Cat3,new_text
0,grocery gourmet food,meat poultry,jerky,description photo product needs changed indica...
1,toys games,games,unknown,great book well thought easily imagine events ...
2,toys games,games,unknown,first year teacher teaching th grade special r...
3,toys games,games,unknown,got book bookfair school looking something sum...
4,toys games,puzzles,jigsaw puzzles,hi im martine redman created puzzle briarpatch...


In [0]:
data1 = data[{"new_text" , "Cat1"}]

In [36]:
data1.head()

,new_text,Cat1
0,description photo product needs changed indica...,grocery gourmet food
1,great book well thought easily imagine events ...,toys games
2,first year teacher teaching th grade special r...,toys games
3,got book bookfair school looking something sum...,toys games
4,hi im martine redman created puzzle briarpatch...,toys games


In [37]:
data1.new_text.apply(lambda x : len(x)).mean()

265.048775

In [38]:
data1.new_text.apply(lambda x : len(x.split())).mean()

40.1735

In [39]:
len(data1.new_text.iloc[5950].split())

156

In [40]:
data1.new_text.iloc[0]

'description photo product needs changed indicate product buffalos version beef jerky'

In [0]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features , split = ' ')
tokenizer.fit_on_texts(data1['new_text'].values)
X = tokenizer.texts_to_sequences(data1['new_text'].values)
X = pad_sequences(X)

In [45]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features , embed_dim , input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out , dropout = 0.2 , recurrent_dropout = 0.2))
model.add(Dense(len(data1['Cat1'].unique()) , activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 396, 128)          256000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 396, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1182      
Total params: 511,982
Trainable params: 511,982
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
Y = pd.get_dummies(data['Cat1']).values
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size = 0.33 , random_state = 42)
print(X_train.shape , Y_train.shape)
print(X_test.shape , Y_test.shape)

(26800, 396) (26800, 6)
(13200, 396) (13200, 6)


In [48]:
batch_size = 32
model.fit(X_train , Y_train , epochs = 7 , batch_size = batch_size , verbose = 2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
 - 723s - loss: 1.0098 - accuracy: 0.6302
Epoch 2/7
 - 705s - loss: 0.6272 - accuracy: 0.7906
Epoch 3/7
 - 639s - loss: 0.5531 - accuracy: 0.8122
Epoch 4/7
 - 655s - loss: 0.5116 - accuracy: 0.8262
Epoch 5/7
 - 634s - loss: 0.4824 - accuracy: 0.8349
Epoch 6/7
 - 632s - loss: 0.4579 - accuracy: 0.8401
Epoch 7/7
 - 634s - loss: 0.4350 - accuracy: 0.8497


In [0]:
valid_size = 1500
X_validate = X_test[-valid_size:]
Y_validate = Y_test[-valid_size:]
X_test = X_test[:-valid_size:]
Y_test = Y_test[:-valid_size:]
score , acc = model.evaluate(X_test , Y_test , verbose = 2 , batch_size = batch_size)

In [50]:
acc

0.8029914498329163

In [59]:
X_validate[0].shape

(396,)

In [98]:
Y_validate[1000]

array([1, 0, 0, 0, 0, 0], dtype=uint8)

In [0]:
out = model.predict(X_validate , verbose = 2)

In [97]:
te = X_validate[1000:1001]
#np.reshape(te , (1,396))
model.predict(te , verbose = 2)

array([[8.9257592e-01, 9.7267440e-04, 3.3920305e-04, 2.7976003e-02,
        6.3793687e-03, 7.1756899e-02]], dtype=float32)